In [1]:
bnp_itemid = [{
  "itemid": "50963",
  "label": "NTproBNP",
  "fluid": "Blood",
  "category": "Chemistry"
}, {
  "itemid": "51921",
  "label": "proBNP, Pleural",
  "fluid": "Pleural",
  "category": "Chemistry"
}]

In [2]:
heart_failure_icd_codes = [{
  "icd_code": "39891",
  "icd_version": "9",
  "long_title": "Rheumatic heart failure (congestive)"
}, {
  "icd_code": "40200",
  "icd_version": "9",
  "long_title": "Malignant hypertensive heart disease without heart failure"
}, {
  "icd_code": "40201",
  "icd_version": "9",
  "long_title": "Malignant hypertensive heart disease with heart failure"
}, {
  "icd_code": "40210",
  "icd_version": "9",
  "long_title": "Benign hypertensive heart disease without heart failure"
}, {
  "icd_code": "40211",
  "icd_version": "9",
  "long_title": "Benign hypertensive heart disease with heart failure"
}, {
  "icd_code": "40290",
  "icd_version": "9",
  "long_title": "Unspecified hypertensive heart disease without heart failure"
}, {
  "icd_code": "40291",
  "icd_version": "9",
  "long_title": "Unspecified hypertensive heart disease with heart failure"
}, {
  "icd_code": "40400",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, malignant, without heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
}, {
  "icd_code": "40401",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, malignant, with heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
}, {
  "icd_code": "40402",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, malignant, without heart failure and with chronic kidney disease stage V or end stage renal disease"
}, {
  "icd_code": "40403",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, malignant, with heart failure and with chronic kidney disease stage V or end stage renal disease"
}, {
  "icd_code": "40410",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, benign, without heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
}, {
  "icd_code": "40411",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, benign, with heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
}, {
  "icd_code": "40412",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, benign, without heart failure and with chronic kidney disease stage V or end stage renal disease"
}, {
  "icd_code": "40413",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, benign, with heart failure and chronic kidney disease stage V or end stage renal disease"
}, {
  "icd_code": "40490",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, unspecified, without heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
}, {
  "icd_code": "40491",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, unspecified, with heart failure and with chronic kidney disease stage I through stage IV, or unspecified"
}, {
  "icd_code": "40492",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, unspecified, without heart failure and with chronic kidney disease stage V or end stage renal disease"
}, {
  "icd_code": "40493",
  "icd_version": "9",
  "long_title": "Hypertensive heart and chronic kidney disease, unspecified, with heart failure and chronic kidney disease stage V or end stage renal disease"
}, {
  "icd_code": "4280",
  "icd_version": "9",
  "long_title": "Congestive heart failure, unspecified"
}, {
  "icd_code": "4281",
  "icd_version": "9",
  "long_title": "Left heart failure"
}, {
  "icd_code": "42820",
  "icd_version": "9",
  "long_title": "Systolic heart failure, unspecified"
}, {
  "icd_code": "42821",
  "icd_version": "9",
  "long_title": "Acute systolic heart failure"
}, {
  "icd_code": "42822",
  "icd_version": "9",
  "long_title": "Chronic systolic heart failure"
}, {
  "icd_code": "42823",
  "icd_version": "9",
  "long_title": "Acute on chronic systolic heart failure"
}, {
  "icd_code": "42830",
  "icd_version": "9",
  "long_title": "Diastolic heart failure, unspecified"
}, {
  "icd_code": "42831",
  "icd_version": "9",
  "long_title": "Acute diastolic heart failure"
}, {
  "icd_code": "42832",
  "icd_version": "9",
  "long_title": "Chronic diastolic heart failure"
}, {
  "icd_code": "42833",
  "icd_version": "9",
  "long_title": "Acute on chronic diastolic heart failure"
}, {
  "icd_code": "42840",
  "icd_version": "9",
  "long_title": "Combined systolic and diastolic heart failure, unspecified"
}, {
  "icd_code": "42841",
  "icd_version": "9",
  "long_title": "Acute combined systolic and diastolic heart failure"
}, {
  "icd_code": "42842",
  "icd_version": "9",
  "long_title": "Chronic combined systolic and diastolic heart failure"
}, {
  "icd_code": "42843",
  "icd_version": "9",
  "long_title": "Acute on chronic combined systolic and diastolic heart failure"
}, {
  "icd_code": "4289",
  "icd_version": "9",
  "long_title": "Heart failure, unspecified"
}, {
  "icd_code": "I0981",
  "icd_version": "10",
  "long_title": "Rheumatic heart failure"
}, {
  "icd_code": "I110",
  "icd_version": "10",
  "long_title": "Hypertensive heart disease with heart failure"
}, {
  "icd_code": "I119",
  "icd_version": "10",
  "long_title": "Hypertensive heart disease without heart failure"
}, {
  "icd_code": "I130",
  "icd_version": "10",
  "long_title": "Hypertensive heart and chronic kidney disease with heart failure and stage 1 through stage 4 chronic kidney disease, or unspecified chronic kidney disease"
}, {
  "icd_code": "I131",
  "icd_version": "10",
  "long_title": "Hypertensive heart and chronic kidney disease without heart failure"
}, {
  "icd_code": "I1310",
  "icd_version": "10",
  "long_title": "Hypertensive heart and chronic kidney disease without heart failure, with stage 1 through stage 4 chronic kidney disease, or unspecified chronic kidney disease"
}, {
  "icd_code": "I1311",
  "icd_version": "10",
  "long_title": "Hypertensive heart and chronic kidney disease without heart failure, with stage 5 chronic kidney disease, or end stage renal disease"
}, {
  "icd_code": "I132",
  "icd_version": "10",
  "long_title": "Hypertensive heart and chronic kidney disease with heart failure and with stage 5 chronic kidney disease, or end stage renal disease"
}, {
  "icd_code": "I50",
  "icd_version": "10",
  "long_title": "Heart failure"
}, {
  "icd_code": "I502",
  "icd_version": "10",
  "long_title": "Systolic (congestive) heart failure"
}, {
  "icd_code": "I5020",
  "icd_version": "10",
  "long_title": "Unspecified systolic (congestive) heart failure"
}, {
  "icd_code": "I5021",
  "icd_version": "10",
  "long_title": "Acute systolic (congestive) heart failure"
}, {
  "icd_code": "I5022",
  "icd_version": "10",
  "long_title": "Chronic systolic (congestive) heart failure"
}, {
  "icd_code": "I5023",
  "icd_version": "10",
  "long_title": "Acute on chronic systolic (congestive) heart failure"
}, {
  "icd_code": "I503",
  "icd_version": "10",
  "long_title": "Diastolic (congestive) heart failure"
}, {
  "icd_code": "I5030",
  "icd_version": "10",
  "long_title": "Unspecified diastolic (congestive) heart failure"
}, {
  "icd_code": "I5031",
  "icd_version": "10",
  "long_title": "Acute diastolic (congestive) heart failure"
}, {
  "icd_code": "I5032",
  "icd_version": "10",
  "long_title": "Chronic diastolic (congestive) heart failure"
}, {
  "icd_code": "I5033",
  "icd_version": "10",
  "long_title": "Acute on chronic diastolic (congestive) heart failure"
}, {
  "icd_code": "I504",
  "icd_version": "10",
  "long_title": "Combined systolic (congestive) and diastolic (congestive) heart failure"
}, {
  "icd_code": "I5040",
  "icd_version": "10",
  "long_title": "Unspecified combined systolic (congestive) and diastolic (congestive) heart failure"
}, {
  "icd_code": "I5041",
  "icd_version": "10",
  "long_title": "Acute combined systolic (congestive) and diastolic (congestive) heart failure"
}, {
  "icd_code": "I5042",
  "icd_version": "10",
  "long_title": "Chronic combined systolic (congestive) and diastolic (congestive) heart failure"
}, {
  "icd_code": "I5043",
  "icd_version": "10",
  "long_title": "Acute on chronic combined systolic (congestive) and diastolic (congestive) heart failure"
}, {
  "icd_code": "I508",
  "icd_version": "10",
  "long_title": "Other heart failure"
}, {
  "icd_code": "I5081",
  "icd_version": "10",
  "long_title": "Right heart failure"
}, {
  "icd_code": "I50810",
  "icd_version": "10",
  "long_title": "Right heart failure, unspecified"
}, {
  "icd_code": "I50811",
  "icd_version": "10",
  "long_title": "Acute right heart failure"
}, {
  "icd_code": "I50812",
  "icd_version": "10",
  "long_title": "Chronic right heart failure"
}, {
  "icd_code": "I50813",
  "icd_version": "10",
  "long_title": "Acute on chronic right heart failure"
}, {
  "icd_code": "I50814",
  "icd_version": "10",
  "long_title": "Right heart failure due to left heart failure"
}, {
  "icd_code": "I5082",
  "icd_version": "10",
  "long_title": "Biventricular heart failure"
}, {
  "icd_code": "I5083",
  "icd_version": "10",
  "long_title": "High output heart failure"
}, {
  "icd_code": "I5084",
  "icd_version": "10",
  "long_title": "End stage heart failure"
}, {
  "icd_code": "I5089",
  "icd_version": "10",
  "long_title": "Other heart failure"
}, {
  "icd_code": "I509",
  "icd_version": "10",
  "long_title": "Heart failure, unspecified"
}, {
  "icd_code": "I9713",
  "icd_version": "10",
  "long_title": "Postprocedural heart failure"
}, {
  "icd_code": "I97130",
  "icd_version": "10",
  "long_title": "Postprocedural heart failure following cardiac surgery"
}, {
  "icd_code": "I97131",
  "icd_version": "10",
  "long_title": "Postprocedural heart failure following other surgery"
}]